In [1]:
import json
import os
import re
from pathlib import Path

import pandas as pd

In [2]:
os.chdir('..')

In [7]:
root_folder = Path("ckpts_arxiv/ppl/PG19")
# root_folder = Path("ckpts_arxiv/ppl/proof_pile")
if not root_folder.exists():
    raise Exception("Path does not exist: {}".format(root_folder))

# get all folders in root_folder
folders = root_folder.iterdir()


# filter using exps
def filter_name(p: Path):
    return p.is_dir()


folders = sorted([f for f in folders if filter_name(f)])
pattern = re.compile(r"^([^|]+).*")
# pattern = re.compile(r"^([^_]+).*")
lines: dict[Path, str] = {path: pattern.match(path.name).group(1) for path in folders}
print(lines)

{PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_16384|Feb15_09-21-51'): 'HS50k_18k_16384', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_2048|Feb14_22-22-17'): 'HS50k_18k_2048', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_4096|Feb14_22-36-05'): 'HS50k_18k_4096', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_8192|Feb14_22-49-18'): 'HS50k_18k_8192', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_i1_16384|Feb15_10-16-13'): 'HS50k_18k_i1_16384', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_i1_2048|Feb15_11-38-29'): 'HS50k_18k_i1_2048', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_i1_4096|Feb15_11-48-21'): 'HS50k_18k_i1_4096', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_i1_8192|Feb15_11-53-33'): 'HS50k_18k_i1_8192', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_relu0_16384|Feb15_11-41-29'): 'HS50k_18k_relu0_16384', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_relu0_2048|Feb15_11-43-39'): 'HS50k_18k_relu0_2048', PosixPath('ckpts_arxiv/ppl/PG19/HS50k_18k_relu0_4096|Feb15_11-50-56'): 'HS50k_18k_relu0_4096', PosixPath('ckpts_arx

In [8]:
data = []
for path, name in lines.items():
    assert path.is_dir(), path
    file_path = path / "result.json"
    if not os.path.isfile(file_path):
        print(f"{file_path} not exists")
        continue
    with open(file_path) as f:
        rst = json.load(f)
    line = {
        "name": name[: -len(str(rst["max_length"])) -1],
        "length": rst["max_length"],
        "ppl": rst["ppl"],
    }
    data.append(line)
df = pd.DataFrame(data)
print(df)

                   name  length        ppl
0             HS50k_18k   16384  30.416761
1             HS50k_18k    2048  18.309721
2             HS50k_18k    4096  18.990034
3             HS50k_18k    8192  18.826849
4          HS50k_18k_i1   16384  31.373378
..                  ...     ...        ...
61  RoPE50k_YaRN8_FT16k    8192  15.423499
62         RoPE50k_YaRN   16384  17.060455
63         RoPE50k_YaRN    2048  14.536722
64         RoPE50k_YaRN    4096  14.449647
65         RoPE50k_YaRN    8192  15.008487

[66 rows x 3 columns]


In [9]:
pivot = df.pivot(index="name", columns="length", values="ppl")
print(pivot)

length                   2048         4096        8192         16384
name                                                                
HS50k_18k            18.309721    18.990034   18.826849    30.416761
HS50k_18k_i1         18.212211    18.776491   19.021738    31.373378
HS50k_18k_relu0      18.043361    18.556450   18.011930    25.850885
HS50k_4k             14.762912    15.315861  404.497354  3224.268142
HS50k_8k             15.711513    15.318255   21.143916   721.723576
NoPE50k              14.613770   326.928595         NaN          NaN
NoPE50k_s1.2         15.004195    15.976459  513.651593  3919.645583
NoPE50k_s1.5         18.965975    20.225210   45.309763   224.096623
NoPE50k_s1.8         30.389720    42.591215   69.083144   198.757643
R2N                  13.365783   106.824878  206.877684   258.866412
R2N_HS18k            16.147798    16.721712   18.547192    23.523664
RoPE1T               12.545147  4150.499868         NaN          NaN
RoPE50k              14.534234   4

In [10]:
with open(str(root_folder) + ".md", "w") as f:
    f.write(pivot.round(2).to_markdown())